In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import os
import zipfile
import pandas as pd

def combine_csv_from_zip_by_year(folder_path, year):
    all_dataframes = []
    zip_files = [f for f in os.listdir(folder_path) if f.endswith('.zip') and str(year) in f]

    print(f"Number of zip files for the year {year}: {len(zip_files)}")

    for zip_filename in zip_files:
        zip_file_path = os.path.join(folder_path, zip_filename)
        with zipfile.ZipFile(zip_file_path, 'r') as z:
            for file_name in z.namelist():
                if file_name.endswith('.csv'):
                    with z.open(file_name) as file:
                        df = pd.read_csv(file)
                        all_dataframes.append(df)

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

folder_path = '/content/drive/MyDrive/Transport_5_years'

year_to_combine = 2023
combined_csv = combine_csv_from_zip_by_year(folder_path, year_to_combine)

if not combined_csv.empty:
    output_directory = '/content/drive/MyDrive/Transport_5_years'
    output_file = f'combined_transport_{year_to_combine}.csv'
    output_path = os.path.join(output_directory, output_file)

    os.makedirs(output_directory, exist_ok=True)

    combined_csv.to_csv(output_path, index=False)
    print(f"The combined CSV file for the year {year_to_combine} has been saved to:", output_path)
else:
    print(f"No CSV files found for the year {year_to_combine}.")


Number of zip files for the year 2023: 12
The combined CSV file for the year 2023 has been saved to: /content/drive/MyDrive/Transport_5_years/combined_transport_2023.csv


In [ ]:
transport_df = pd.read_csv('/content/drive/MyDrive/data/Transport_5_years/combined_transport_2019.csv')
bird_strike_df = pd.read_excel('/content/drive/MyDrive/data/bird_5_years/2019.xlsx')

<ipython-input-37-07b2fea11ae9>:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  transport_df = pd.read_csv('/content/drive/MyDrive/data/Transport_5_years/combined_transport_2019.csv')


In [ ]:
# Standardizing Airport codes

data = ""
with open("/content/drive/MyDrive/AirCode.txt", "r") as f:
  data = f.read()
  f.close()

data = data.split("\n")
airport_codes = {}
for d in data:
  temp = d.split(":")
  airport_codes[temp[0]] = temp[1]

airlines = pd.read_csv('/content/drive/MyDrive/airlines.csv')
airports = pd.read_csv('/content/drive/MyDrive/airports.csv')
airports = airports.rename(columns={'IATA_CODE':'ORIGIN'})
airlines = airlines.rename(columns={'IATA_CODE':'OP_UNIQUE_CARRIER'})

In [ ]:
bird_strike_df = bird_strike_df.replace({"AIRPORT_ID": airport_codes})
transport_df = transport_df.merge(airports, on='ORIGIN')
transport_df = transport_df.merge(airlines, on='OP_UNIQUE_CARRIER')

In [ ]:
# Convert date formats to match
transport_df['FL_DATE'] = pd.to_datetime(transport_df['FL_DATE'])
bird_strike_df['INCIDENT_DATE '] = pd.to_datetime(bird_strike_df['INCIDENT_DATE '])

# Standardize airport codes
transport_df['FL_NUMBER'] = transport_df['FL_NUMBER'].astype(str).str.strip().str.upper()
bird_strike_df['FLT'] = bird_strike_df['FLT'].str.strip().str.upper()

# Standardize airport codes
transport_df['ORIGIN'] = transport_df['ORIGIN'].str.strip().str.upper()
transport_df['DEST'] = transport_df['DEST'].str.strip().str.upper()
bird_strike_df['AIRPORT_ID'] = bird_strike_df['AIRPORT_ID'].str.strip().str.upper()

# Merge the datasets using Airport Code and Date as the merge keys
merged_df_source = pd.merge(transport_df, bird_strike_df,
                     left_on=["ORIGIN", "FL_DATE", "OP_CARRIER_FL_NUM"],
                     right_on=["AIRPORT_ID", "INCIDENT_DATE ", "FLT"],
                     how="inner")

merged_df_dest = pd.merge(transport_df, bird_strike_df,
                     left_on=["DEST", "FL_DATE", "OP_CARRIER_FL_NUM"],
                     right_on=["AIRPORT_ID", "INCIDENT_DATE ", "FLT"],
                     how="inner")
merged_df = pd.concat([merged_df_source, merged_df_dest], ignore_index=True)
if merged_df.empty:
    print("No matching records found.")
else:
    # Post-merge processing
    # Handle missing values, create new features, etc.
    merged_df.fillna(0, inplace=True)

    # Save the merged csv
    merged_df.to_csv("/content/drive/MyDrive/SD/merged_2019.csv", index=False)

<ipython-input-39-747fb20dc696>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transport_df['FL_DATE'] = pd.to_datetime(transport_df['FL_DATE'])


In [ ]:
merged_2023_df = pd.read_csv('/content/drive/MyDrive/SD/merged_2019.csv')
print(len(merged_2023_df))

6087


<ipython-input-40-f2f5a46c2153>:1: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_2023_df = pd.read_csv('/content/drive/MyDrive/SD/merged_2019.csv')


In [ ]:
# Merging all CSVs

import pandas as pd

file_paths = [
    '/content/drive/MyDrive/SD/merged_2019.csv',
    '/content/drive/MyDrive/SD/merged_2020.csv',
    '/content/drive/MyDrive/SD/merged_2021.csv',
    '/content/drive/MyDrive/SD/merged_2022.csv',
    '/content/drive/MyDrive/SD/merged_2023.csv'
]

dataframes = [pd.read_csv(file) for file in file_paths]

combined_csv = pd.concat(dataframes, ignore_index=True)

output_path = '/content/drive/MyDrive/SD/combined_all_SD.csv'

combined_csv.to_csv(output_path, index=False)

print(f"The combined CSV file has been saved to: {output_path}")


<ipython-input-41-5566e053f338>:11: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]
<ipython-input-41-5566e053f338>:11: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]
<ipython-input-41-5566e053f338>:11: DtypeWarning: Columns (60,134) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]


The combined CSV file has been saved to: /content/drive/MyDrive/SD/combined_all_SD.csv


In [ ]:
combined_df = pd.read_csv('/content/drive/MyDrive/SD/combined_all_SD.csv')
print(len(combined_df))

27319


<ipython-input-42-020709d41a34>:1: DtypeWarning: Columns (60,134) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('/content/drive/MyDrive/SD/combined_all_SD.csv')


In [ ]:
print(len(combined_df))

10987


In [ ]:
print(combined_df.columns)

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       ...
       'NR_INJURIES', 'NR_FATALITIES', 'COMMENTS', 'REPORTER_NAME',
       'REPORTER_TITLE', 'SOURCE', 'PERSON', 'LUPDATE', 'IMAGE', 'TRANSFER'],
      dtype='object', length=149)


In [ ]:
print(combined_df.head)

<bound method NDFrame.head of        YEAR  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE OP_UNIQUE_CARRIER  \
0      2019     10             2            3  2019-10-02                AA   
1      2019     10             2            3  2019-10-02                DL   
2      2019     10             5            6  2019-10-05                AA   
3      2019     10             5            6  2019-10-05                DL   
4      2019     10             7            1  2019-10-07                AA   
...     ...    ...           ...          ...         ...               ...   
10982  2023      1             7            6  2023-01-07                HA   
10983  2023      1             7            6  2023-01-07                HA   
10984  2023      7             5            3  2023-07-05                HA   
10985  2023      4             5            3  2023-04-05                HA   
10986  2023      9            21            4  2023-09-21                HA   

       OP_CARRIER_AIR

In [ ]:
import pandas as pd

file_paths = [
    '/content/drive/MyDrive/SD/merged_2019.csv',
    '/content/drive/MyDrive/SD/merged_2020.csv',
    '/content/drive/MyDrive/SD/merged_2021.csv',
    '/content/drive/MyDrive/SD/merged_2022.csv',
    '/content/drive/MyDrive/SD/merged_2023.csv'
]

dataframes = [pd.read_csv(file) for file in file_paths]

combined_csv = pd.concat(dataframes, ignore_index=True)

output_path = '/content/drive/MyDrive/SD/combined_all_SD.csv'

combined_csv.to_csv(output_path, index=False)

print(f"The combined CSV file has been saved to: {output_path}")


In [ ]:
# /content/drive/MyDrive/data/Transport_5_years/Sep-2019.zip

import os
import zipfile
import pandas as pd

def combine_csv_from_zip_by_year(folder_path, month):
    all_dataframes = []
    zip_files = [f for f in os.listdir(folder_path) if f.endswith('.zip') and str(month) in f]
    print(zip_files)
    for zip_filename in zip_files:
        zip_file_path = os.path.join(folder_path, zip_filename)
        with zipfile.ZipFile(zip_file_path, 'r') as z:
            for file_name in z.namelist():
                if file_name.endswith('.csv'):
                    with z.open(file_name) as file:
                        df = pd.read_csv(file)
                        all_dataframes.append(df)

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

folder_path = '/content/drive/MyDrive/data/Transport_5_years'

month = 'Sep'
combined_csv = combine_csv_from_zip_by_year(folder_path, month)

if not combined_csv.empty:
    output_directory = '/content/drive/MyDrive/data/Transport_5_years'
    output_file = f'combined_transport_{month}.csv'
    output_path = os.path.join(output_directory, output_file)

    os.makedirs(output_directory, exist_ok=True)

    combined_csv.to_csv(output_path, index=False)
    print(f"The combined CSV file for the year {month} has been saved to:", output_path)
else:
    print(f"No CSV files found for the year {month}.")


['Sep-2019.zip', 'Sep-2021.zip', 'Sep-2022.zip', 'Sep-2023.zip', 'Sep-2020.zip']
The combined CSV file for the year Sep has been saved to: /content/drive/MyDrive/data/Transport_5_years/combined_transport_Sep.csv
